## Clean Kaggle Data
### converting strings to correct data

In [1]:
import json
import pandas as pd
import numpy as np
import re

In [2]:
kaggle_metadata = pd.read_csv('../movies_metadata.csv', low_memory=False)
ratings = pd.read_csv('../ratings.csv')

In [3]:
kaggle_metadata.sample(n=2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
13769,False,NaN,1000000,"[{'id': 18, 'name': 'Drama'}, {'id': 14, 'name...",http://www.specialthemovie.com/,13856,tt0479162,en,Special,A lonely metermaid has a psychotic reaction to...,...,2006-01-30,7202.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Special,False,6.6,32.0
4439,False,"{'id': 91945, 'name': 'American Ninja Collecti...",0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,25682,tt0096804,fr,American Ninja 3: Blood Hunt,"Jackson is back, and now he has a new partner,...",...,1989-02-24,0.0,89.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,American Ninja 3: Blood Hunt,False,4.1,33.0


In [4]:
ratings.sample(n=5)

,userId,movieId,rating,timestamp
2269479,23564,1148,3.0,1190320211
4151825,43128,586,3.5,1203885237
11493596,119299,110,4.5,1208709809
14000475,145495,673,3.5,1486647406
19893423,206850,2568,2.0,1047317339


In [5]:
kaggle_metadata.dtypes

adult                     object
belongs_to_collection     object
budget                    object
genres                    object
homepage                  object
id                        object
imdb_id                   object
original_language         object
original_title            object
overview                  object
popularity                object
poster_path               object
production_companies      object
production_countries      object
release_date              object
revenue                  float64
runtime                  float64
spoken_languages          object
status                    object
tagline                   object
title                     object
video                     object
vote_average             float64
vote_count               float64
dtype: object